In [13]:
%pip install --upgrade tiktoken
%pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 9.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd
import numpy as np
import tiktoken

In [33]:
mock_data_path = "../../data/llm-anamnesis/mock_data_2_big.xlsx"
dfs = pd.read_excel(mock_data_path, sheet_name=None, index_col=None)

# Convert json objects to dict
#for table, df in dfs.items():
#    for col in ["schedule_params", "filter", "model_params", "metadata"]:
#        if col in df.columns:
#            df[col] = df[col].apply(lambda x: json.loads(x))

raw_df = dfs["raw"]
config_df = dfs["config"]
summary_df = dfs["summarized"]

In [34]:
summary_df['patient_id'] = summary_df['patient_ids'].apply(lambda x: x[1:-1] if x[0]=='[' else x)
summary_df['date'] = summary_df['dates'].apply(lambda x:x[1:-1])
raw_df['patient_id'] = raw_df['patient_id'].astype(str) 
all_text = ""
#pandas_list = list()
answers = list()
references = list()
nAnswers = 0

texts = []

for index,r in summary_df.iterrows():
    temp_df = raw_df.loc[(raw_df['patient_id']==r['patient_id'])&(raw_df['date']==r['date'])]
    text = ""
    for row in temp_df.iterrows():
        row = row[1]
        if row['metadata'] != np.nan:
            text += str(row['metadata'])
        text +=row['text']
    if text == "":
        continue
    
    texts.append(text)

In [35]:
def mean_tokens_from_list(str_list : [str]):
    encoding = tiktoken.encoding_for_model("gpt-4-turbo-preview")
    return np.mean([len(encoding.encode(elem)) for elem in str_list])


In [36]:
mean_tokens_from_list(texts) #counts the avg number of input tokens (just using patient-info)

674.5909090909091

In [37]:
mean_tokens_from_list(summary_df["text"].to_list()) # counts the avg number of output tokens

304.7916666666667

In [38]:
sample_instruction = """You are a helpful medical assistant who helps medical proffesionals by summarizing information about patients.
Answer with a bulleted list for each category in the given template.
Answer in Swedish.
<</SYS>>
Below is the history of a patient during one day
<anamnesis>
*
</anamnesis>
You must select information that fits in the template below. Avoid unnecessary information and only pick out things that relate to each heading. If relevant information is missing, leave the heading blank. Do not include any information in multiple headings.
<template>
*Sjukdomshistoria (The patient's diagnoses, medical history and risk factors (e.g. diseases in the family))*

*Sökorsaker (Patient's symptoms and/or date of procedure)*

*Åtgärder (Planned examinations, treatments and measures)*

</template>"""
len(encoding.encode(sample_instruction))

176

In [39]:
# input cost per example:
# assumption: instruction is 200 tokens
# total input tokens = instruction + input + output = 675 + 305 + 200 = 1180
# input cost per example = 1180 * 1/100_000 = 0.0118

# output cost per example:
# total output tokens = input + output = 980
# output cost per example = 980 * 3/100_000 = 0.0294

#total cost per example:
# input cost + output cost = 0.0118 + 0.0294 = 0.0412

# total cost for 1000 examples:
# 0.0412 * 1000 = 41.2$